In [ ]:
from mne.stats import permutation_cluster_1samp_test
import numpy as np

import pickle

import os
os.chdir('..')
from _parameters import *

In [ ]:
def load_tfr(filename):

    cvsi_avg_f = open(dirs['cvsi'] + '/' + filename, 'rb')
    cvsi_dat, cvsi_avg = pickle.load(cvsi_avg_f)
    cvsi_avg_f.close()

    return cvsi_dat, cvsi_avg

def load_tc(filename):

    cvsi_avg_f = open(dirs['cvsi'] + '/' + filename, 'rb')
    cvsi_dat = pickle.load(cvsi_avg_f)
    cvsi_avg_f.close()
    
    return cvsi_dat

In [ ]:
def stat_TFR(data):

    data = np.asarray(data)

    # Run cluster perm
    t, clust, p, h0 = permutation_cluster_1samp_test(data, n_permutations=10000, out_type= 'mask')

    # Create mask
    mask = np.asarray(clust)[p<.05]
    mask = np.kron(np.squeeze(mask), np.ones((10,10)))
    
    stat = {
        'mask': mask,
        'sig': p
    }

    return stat

In [ ]:
def stat_tc(data,time):

    t, clust, p, h0 = permutation_cluster_1samp_test(data, n_permutations=10000)

    # Get t-index of sign clusters
    sig_index = np.asarray(clust, dtype='object')[p<.05]

    # Create mask
    mask = []
    for i in range(len(sig_index)):
        mask = np.append(mask, (time[sig_index[i][0]]))

    return mask

In [ ]:
def run_stat_TFR(filename, save = False):
    
        cvsi_dat = load_tfr(filename)
    
        stats_TFR = {c:None for c in cvsi_dat.keys()}
    
        # Run for each cond
        for c in cvsi_dat.keys():
    
            tfr_dat = cvsi_dat[c]
    
            # Add stats to dict
            stats_TFR[c] = stat_TFR(tfr_dat)
    
        if save:
            stat_tfr_f = open(dirs['stat'] + '/' + filename.replace('cvsi', 'stat'), 'wb')
            pickle.dump(stats_TFR, stat_tfr_f)
            stat_tfr_f.close()

        return stats_TFR

def run_stat_tc(filename, save = False):
    
        cvsi_tc = load_tc(filename)
    
        stats_tc = {c:None for c in cvsi_tc.keys() if c != 'time'}
        time = cvsi_tc['time']

        # Run for each cond
        for c in stats_tc.keys():
            print('Running stats for ' + c)
    
            tc_dat = cvsi_tc[c]
    
            # Add stats to dict
            stats_tc[c] = stat_tc(tc_dat, time)
    
        if save:
            stat_tc_f = open(dirs['stat'] + '/' + filename.replace('cvsi', 'stat'), 'wb')
            pickle.dump(stats_tc, stat_tc_f)
            stat_tc_f.close()

        return stats_tc


In [ ]:
# Run stats

stats_tc = run_stat_tc(filename = 'cvsi_tc_enc1_samerev.pkl', save=True)


In [ ]:
# Stats behavior split

cvsi_tc = load_tc('cvsi_tc_enc1_err.pkl')

conds = ['mot/same', 'mot/rvrs', 'vis/same', 'vis/rvrs']
cvsi_trade = {c:None for c in conds}
stats_trade = {c:None for c in conds}

trade = ['err_trade_low', 'err_trade_high']

for c in conds:
    t1, t2 = trade
    cvsi_trade[c] = cvsi_tc[c+'/'+t1] - cvsi_tc[c+'/'+t2]

time = cvsi_tc['time']

# Run for each cond
for c in stats_trade.keys():
    print('Running stats for ' + c)

    tc_dat = cvsi_trade[c]

    # Add stats to dict
    stats_trade[c] = stat_tc(tc_dat, time)


In [ ]:
cvsi_trade['vis/rvrs'].shape